In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
# do train_test_split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=0) 


In [7]:
X_train.head()

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore


In [8]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [9]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

# 1. Without using Column Transformer

In [11]:
# Adding simple imputer to fever column (because it contains missing values)

si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']]) 

# for test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape    

(80, 1)

In [12]:
# ordinal encoding to cough column

oe = OrdinalEncoder(categories = [['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# for test data 
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [13]:
# one hot encoding to 'gender' and 'city' column

ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# for test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [14]:
# now, for extracting "age" from X_train

X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# for test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values 

X_train_age.shape

(80, 1)

In [15]:
# Now, we have to add all these columns and concatenate

X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough), axis=1)

# for test data
X_test_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough), axis=1)

X_train_transformed.shape

(80, 7)

# 2. With using Column Transformer

In [16]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers = [
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False,drop='first'), ['gender','city'])
], remainder='passthrough')                                                    

In [17]:
transformer.fit_transform(X_train).shape   

(80, 7)

In [18]:
transformer.fit_transform(X_test).shape       

(20, 7)